In [20]:
!pip install "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1 torch==2.7.0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [1]:
def get_default_embedding_model():
    """
    获取默认的嵌入模型
    """
    from pymilvus import model as milvus_model
    embedding_model = milvus_model.DefaultEmbeddingFunction()
    test_embedding = embedding_model.encode_queries(["test"])
    embedding_dim = len(test_embedding[0])

    from pymilvus import MilvusClient
    milvus_client = MilvusClient(uri="./mfd.db")

    return embedding_model, embedding_dim, milvus_client


embedding_model, embedding_dim, milvus_client = get_default_embedding_model()

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.11/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [2]:
collection_name = "mfd_collection"


def create_milvus_collection():

    if milvus_client.has_collection(collection_name):
        milvus_client.drop_collection(collection_name)

    milvus_client.create_collection(
        collection_name=collection_name,
        dimension=embedding_dim,
        metric_type="IP",  # 内积距离
        consistency_level="Strong",  # 一致性相关
    )
    print(f"Created milvus collection: {collection_name}")


create_milvus_collection()

Created milvus collection: mfd_collection


In [4]:
mfd_file_path = "mfd.md"


def insert_data_from_files():
    text_lines = []
    file_paths = [mfd_file_path]
    for file_path in file_paths:
        with open(file_path, "r") as file:
            file_text = file.read()
        text_lines += file_text.split("# ")

    data = []
    doc_embeddings = embedding_model.encode_documents(text_lines)

    for i, line in enumerate(text_lines):
        data.append({"id": i, "vector": doc_embeddings[i], "text": line})
        print(f"creating embeddings: {i}, {line[:min(len(line),20)]}...")

    milvus_client.insert(collection_name=collection_name, data=data)
    print("mfd.md is inserted to db")


insert_data_from_files()

creating embeddings: 0, #...
creating embeddings: 1, 中华人民共和国民法典

##...
creating embeddings: 2, （二）物权编

###...
creating embeddings: 3, 第一章 一般规定

**第二百零四条**...
creating embeddings: 4, 第二章 所有权

####...
creating embeddings: 5, 第一节 一般规定

**第二百四十四条*...
creating embeddings: 6, 第二节 共有

**第三百一十条** 两...
creating embeddings: 7, 第三节 相邻关系

**第三百二十六条*...
creating embeddings: 8, 第四节 所有权取得的特别规定

**第三...
creating embeddings: 9, 第三章 用益物权

####...
creating embeddings: 10, 第一节 一般规定

**第三百五十三条*...
creating embeddings: 11, 第二节 土地承包经营权

**第三百五十...
creating embeddings: 12, 第三节 建设用地使用权

**第三百六十...
creating embeddings: 13, 第四节 宅基地使用权

**第三百七十七...
creating embeddings: 14, 第五节 居住权

**第三百八十二条**...
creating embeddings: 15, 第六节 地役权

**第三百八十八条**...
creating embeddings: 16, 第四章 担保物权

####...
creating embeddings: 17, 第一节 一般规定

**第三百九十九条*...
creating embeddings: 18, 第二节 抵押权

**第四百零九条** ...
creating embeddings: 19, 第三节 质权

#####...
creating embeddings: 20, 一、动产质权

**第四百三十四条** ...
creating embeddings: 21, 二、权利质权

**第四百四十九

In [19]:
question = "业主有哪些权利与义务"
search_params = {
    "metric_type": "IP",   # 余弦相似度
    "params": {"nprobe": 16}   # 搜索的聚类中心数量，增大可提高召回率但降低性能
}


def search_milvus_db():
    search_res = milvus_client.search(
        collection_name=collection_name,
        data=embedding_model.encode_queries(
            [question]
        ),  # 将问题转换为嵌入向量
        limit=5,  # 返回前5个结果
        search_params=search_params,
        output_fields=["text"],  # 返回 text 字段
    )
    return search_res


search_res = search_milvus_db()

In [23]:
import json
search_res[0]
retrievals_with_distance = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrievals_with_distance, indent=4, ensure_ascii=False))


[
    [
        "第三章 合同的变更和转让\n\n**第五百四十八条** 当事人协商一致，可以变更合同。\n\n**第五百四十九条** 当事人对合同变更的内容约定不明确的，推定为未变更。\n\n**第五百五十条** 债权人可以将合同的权利全部或者部分转让给第三人，但是有下列情形之一的除外：\n（一）根据合同性质不得转让；\n（二）按照当事人约定不得转让；\n（三）依照法律规定不得转让。\n债权人转让权利的，应当通知债务人。未经通知，该转让对债务人不发生效力。\n\n**第五百五十一条** 债权人转让权利的，受让人取得与债权有关的从权利，但是该从权利专属于债权人自身的除外。\n\n**第五百五十二条** 债务人将合同的义务全部或者部分转让给第三人的，应当经债权人同意。\n\n**第五百五十三条** 债务人转让义务的，新债务人可以主张原债务人对债权人的抗辩。\n新债务人承担债务的，应当承担与主债务有关的从债务，但是该从债务专属于原债务人自身的除外。\n\n**第五百五十四条** 当事人一方经对方同意，可以将自己在合同中的权利和义务一并转让给第三人。\n\n**第五百五十五条** 权利和义务一并转让的，适用债权转让、债务转让的有关规定。\n\n**第五百五十六条** 合同变更的，不影响当事人请求损害赔偿的权利。\n\n###",
        0.6202714443206787
    ],
    [
        "第五章 占有\n\n**第四百七十一条** 占有是指对物事实上的控制和支配。\n\n**第四百七十二条** 占有可以分为直接占有和间接占有。\n直接占有是指直接对物进行控制和支配。\n间接占有是指通过他人对物进行控制和支配。\n\n**第四百七十三条** 占有的取得和消灭，适用本法有关物权设立和消灭的规定。\n\n**第四百七十四条** 占有人合法占有动产的，善意取得人取得该动产所有权。\n占有人非法占有动产的，善意取得人取得该动产所有权。\n\n**第四百七十五条** 占有物毁损、灭失的，占有人应当承担赔偿责任。\n占有人善意占有物的，不承担赔偿责任。\n占有人恶意占有物的，应当承担赔偿责任。\n\n**第四百七十六条** 占有被侵夺的，占有人有权请求返还原物。\n占有物毁损、灭失的，占有人有权请求赔偿损失。\n\n**第四百七十七

In [25]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrievals_with_distance]
)
print(context)

第三章 合同的变更和转让

**第五百四十八条** 当事人协商一致，可以变更合同。

**第五百四十九条** 当事人对合同变更的内容约定不明确的，推定为未变更。

**第五百五十条** 债权人可以将合同的权利全部或者部分转让给第三人，但是有下列情形之一的除外：
（一）根据合同性质不得转让；
（二）按照当事人约定不得转让；
（三）依照法律规定不得转让。
债权人转让权利的，应当通知债务人。未经通知，该转让对债务人不发生效力。

**第五百五十一条** 债权人转让权利的，受让人取得与债权有关的从权利，但是该从权利专属于债权人自身的除外。

**第五百五十二条** 债务人将合同的义务全部或者部分转让给第三人的，应当经债权人同意。

**第五百五十三条** 债务人转让义务的，新债务人可以主张原债务人对债权人的抗辩。
新债务人承担债务的，应当承担与主债务有关的从债务，但是该从债务专属于原债务人自身的除外。

**第五百五十四条** 当事人一方经对方同意，可以将自己在合同中的权利和义务一并转让给第三人。

**第五百五十五条** 权利和义务一并转让的，适用债权转让、债务转让的有关规定。

**第五百五十六条** 合同变更的，不影响当事人请求损害赔偿的权利。

###
第五章 占有

**第四百七十一条** 占有是指对物事实上的控制和支配。

**第四百七十二条** 占有可以分为直接占有和间接占有。
直接占有是指直接对物进行控制和支配。
间接占有是指通过他人对物进行控制和支配。

**第四百七十三条** 占有的取得和消灭，适用本法有关物权设立和消灭的规定。

**第四百七十四条** 占有人合法占有动产的，善意取得人取得该动产所有权。
占有人非法占有动产的，善意取得人取得该动产所有权。

**第四百七十五条** 占有物毁损、灭失的，占有人应当承担赔偿责任。
占有人善意占有物的，不承担赔偿责任。
占有人恶意占有物的，应当承担赔偿责任。

**第四百七十六条** 占有被侵夺的，占有人有权请求返还原物。
占有物毁损、灭失的，占有人有权请求赔偿损失。

**第四百七十七条** 占有被侵夺的，占有人有权请求返还原物。
占有物毁损、灭失的，占有人有权请求赔偿损失。

**第四百七十八条** 占有物毁损、灭失的，占有人应当承担赔偿责任。
占有人善意占有物的，不承担赔偿责任。
占有人恶意占有物的，应

In [29]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。
<context>
{context}
</context>
<question>
{question}
</question>
"""
print(USER_PROMPT)


请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。
<context>
第三章 合同的变更和转让

**第五百四十八条** 当事人协商一致，可以变更合同。

**第五百四十九条** 当事人对合同变更的内容约定不明确的，推定为未变更。

**第五百五十条** 债权人可以将合同的权利全部或者部分转让给第三人，但是有下列情形之一的除外：
（一）根据合同性质不得转让；
（二）按照当事人约定不得转让；
（三）依照法律规定不得转让。
债权人转让权利的，应当通知债务人。未经通知，该转让对债务人不发生效力。

**第五百五十一条** 债权人转让权利的，受让人取得与债权有关的从权利，但是该从权利专属于债权人自身的除外。

**第五百五十二条** 债务人将合同的义务全部或者部分转让给第三人的，应当经债权人同意。

**第五百五十三条** 债务人转让义务的，新债务人可以主张原债务人对债权人的抗辩。
新债务人承担债务的，应当承担与主债务有关的从债务，但是该从债务专属于原债务人自身的除外。

**第五百五十四条** 当事人一方经对方同意，可以将自己在合同中的权利和义务一并转让给第三人。

**第五百五十五条** 权利和义务一并转让的，适用债权转让、债务转让的有关规定。

**第五百五十六条** 合同变更的，不影响当事人请求损害赔偿的权利。

###
第五章 占有

**第四百七十一条** 占有是指对物事实上的控制和支配。

**第四百七十二条** 占有可以分为直接占有和间接占有。
直接占有是指直接对物进行控制和支配。
间接占有是指通过他人对物进行控制和支配。

**第四百七十三条** 占有的取得和消灭，适用本法有关物权设立和消灭的规定。

**第四百七十四条** 占有人合法占有动产的，善意取得人取得该动产所有权。
占有人非法占有动产的，善意取得人取得该动产所有权。

**第四百七十五条** 占有物毁损、灭失的，占有人应当承担赔偿责任。
占有人善意占有物的，不承担赔偿责任。
占有人恶意占有物的，应当承担赔偿责任。

**第四百七十六条** 占有被侵夺的，占有人有权请求返还原物。
占有物毁损、灭失的，占有人有权请求赔偿损失。

**第四百七十七条** 占有被侵夺的，占有人有权请求返还原物。
占有物毁损、灭失的，占有人有权请求赔偿损失。

*

In [30]:
def ask_llm(user_msg, sys_msg=""):
    reasoning_content = ""  # 定义完整思考过程
    answer_content = ""     # 定义完整回复
    is_answering = False   # 判断是否结束思考过程并开始回复

    # 从环境变量获取 Dashscope API Key
    import os
    api_key = os.getenv("DASHSCOPE_API_KEY")
    if not api_key:
        raise ValueError("请设置 DASHSCOPE_API_KEY 环境变量")

    # 初始化 OpenAI 客户端
    from openai import OpenAI
    client = OpenAI(
        api_key=api_key,
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    )

    # 创建聊天完成请求
    completion = client.chat.completions.create(
        model="deepseek-r1",  # 此处以 deepseek-r1 为例，可按需更换模型名称
        messages=[
            {"role": "system", "content": sys_msg},
            {"role": "user", "content": user_msg}
        ],
        stream=True,
        # 解除以下注释会在最后一个chunk返回Token使用量
        # stream_options={
        #     "include_usage": True
        # }
    )

    print("\n" + "=" * 20 + "思考过程" + "=" * 20 + "\n")

    for chunk in completion:
        # 如果chunk.choices为空，则打印usage
        if not chunk.choices:
            print("\nUsage:")
            print(chunk.usage)
        else:
            delta = chunk.choices[0].delta
            # 打印思考过程
            if hasattr(delta, 'reasoning_content') and delta.reasoning_content != None:
                print(delta.reasoning_content, end='', flush=True)
                reasoning_content += delta.reasoning_content
            else:
                # 开始回复
                if delta.content != "" and is_answering == False:
                    print("\n" + "=" * 20 + "完整回复" + "=" * 20 + "\n")
                    is_answering = True
                # 打印回复过程
                print(delta.content, end='', flush=True)
                answer_content += delta.content
    return answer_content


answer_content = ask_llm(USER_PROMPT, SYSTEM_PROMPT)


====================思考过程====================

首先，问题是：“业主有哪些权利与义务？”我需要从提供的上下文段落中找到答案。上下文是关于中国民法典的条款，涉及合同、占有、地役权、担保物权、土地承包经营权等。

问题中的“业主”可能指的是“业主”或“所有权人”，但在上下文中，没有直接提到“业主”。我需要检查上下文是否涉及业主的权利和义务。

回顾上下文：

- 第三章：合同的变更和转让 – 这涉及合同当事人的权利和义务，但未指定业主。

- 第五章：占有 – 这涉及占有人的权利和义务，如占有的定义、取得、消灭、赔偿责任等。占有可以是业主的占有，但占有不一定是所有权。

- 第六节：地役权 – 这涉及地役权人和供役地权利人的权利和义务。供役地权利人可能是业主。

- 第一节：一般规定（担保物权） – 这涉及担保物权人的权利和义务，但未直接涉及业主。

- 第二节：土地承包经营权 – 这涉及土地承包经营权人的权利和义务，但土地承包经营权是使用权，不是所有权。业主可能指土地所有者。

在上下文中，最相关的是“地役权”部分，因为它涉及不动产的所有者（业主）的权利和义务。

具体看地役权部分：

- **第三百八十八条**：定义地役权，地役权人有权利用他人的不动产（供役地）以提高自己不动产（需役地）的效益。供役地权利人就是业主。

- **第三百八十九条**：地役权合同的内容。

- **第三百九十条**：设立地役权需登记。

- **第三百九十一条**：地役权人行使权利不得损害供役地权利人的合法权益。供役地权利人（业主）有权不受损害。

- **第三百九十二条**：供役地权利人应当按照约定允许地役权人利用其土地，不得妨碍地役权人行使权利。这是业主的义务。

- **第三百九十三条**：地役权消灭的情形。

- **第三百九十四条**：地役权人支付报酬的义务。

- **第三百九十五条**：地役权不得单独转让，需与相关权利一并转让。

- **第三百九十六条**：供役地转让、出租或抵押不影响地役权。

- **第三百九十七条**：如果地役权因供役地转让等消灭，地役权人有权请求赔偿。

- **第三百九十八条**：类似388条。

从地役权部分，供役地权利人（即业主）的权利和义务包括：

- 义务：按照约定允许地役权人利用土地（第39

In [31]:
# 最后把答案写入house_owner.md

with open("house_owner.md", "w", encoding="utf-8") as f:
    f.write(answer_content)
print("业主相关权利与义务以归纳写入house_owner.md")

业主相关权利与义务以归纳写入house_owner.md
